In [ ]:
import os
from dotenv import load_dotenv
from azure.ai.projects import AIProjectClient
from azure.ai.agents.models import CodeInterpreterTool
from azure.identity import DefaultAzureCredential

In [5]:
load_dotenv(override=True)

AIF_PROJECT_ENDPOINT = os.getenv("AIF_PROJECT_ENDPOINT")
MODEL_GPT4O = os.getenv("MODEL_GPT4O")

project_client = AIProjectClient(
  endpoint=AIF_PROJECT_ENDPOINT,
  credential=DefaultAzureCredential()
) 

In [6]:
AIF_PROJECT_ENDPOINT

'https://ai-base-prj-01-20250718.services.ai.azure.com/api/projects/ai-base-prj-01'

### Define Agent

In [7]:
agent = project_client.agents.create_agent(
  model=MODEL_GPT4O,
  name="test-simple-agent",
  instructions="You are and AI Agent that only speaks using emojis. You will only respond using emojis.",
)

print(f"Created agent with ID: {agent.id}")

Created agent with ID: asst_LzDQWo1wXUbrwsGWJMtk3whp


In [8]:
thread = project_client.agents.threads.create()
print(f"Created thread with ID: {thread.id}")

Created thread with ID: thread_qlPB8cXQU9gldQ3AZmX7vAGE


In [9]:
def send_message(thread_id, message):
  message = project_client.agents.messages.create(
    thread_id=thread_id,
    role="user",
    content=message
  )

In [ ]:
def run_agent(thread_id, agent_id):
  response = project_client.agents.runs.create_and_process(
    thread_id=thread_id,
    agent_id=agent_id
  )

  print(f"Status: {response.status}")

  if response.status == "failed":
    raise Exception(f"Agent run failed: {response.last_error}")

  return True

In [28]:
def get_messages(thread_id):
  messages = project_client.agents.messages.list(
    thread_id=thread_id
  )

  msg = []
  for m in list(messages):
    msg.append({
      "role": m["role"],
      "content": m.content[0]["text"]["value"]
    })

  return msg

### Demo

In [36]:
while True:
  user_msg = input("USER >>> ")
  
  if user_msg.lower() == "exit":
    break
  
  send_message(thread.id, user_msg)
  run_agent(thread.id, agent.id)
  messages = list(get_messages(thread.id))
  
  last_message = messages[0] if messages else None

  #print(last_message)
  print(f"{last_message['role']}: {last_message['content']}")

  print("------------------------\n\n")

Status: RunStatus.COMPLETED
assistant: 👋😃
------------------------




In [ ]:
import gradio as gr

with gr.Blocks() as demo:

  def get_ai_response(message, history):
    send_message(thread.id, user_msg)
    run_agent(thread.id, agent.id)
    messages = list(get_messages(thread.id))
    
    return messages[0]["content"]
  
  gr.Markdown("## Azure AI Agent Test")
  gr.ChatInterface(
    fn=get_ai_response,
    type="messages"
  )

demo.launch(height=600, inline=True)

c:\dev\azure-ai-agent\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.


Status: RunStatus.COMPLETED
Status: RunStatus.COMPLETED


### Clean up

In [15]:
project_client.agents.delete_agent(agent.id)
print(f"Deleted agent with ID: {agent.id}")

Deleted agent with ID: asst_iYKgPRUBzAvSlHhQANGuoUVv
